In [1]:
# imports
import networkx as nx
import matplotlib.pyplot as plt
import random
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings("ignore")
import sys

In [2]:
NUM_NETWORKS = 100
NUM_EVENTS = 500
GAMMA=1
P = 1

In [3]:
def isComplete(G):
    n = len(G.nodes())
    return len(G.edges) == (n*(n-1))/2

In [4]:
def choose(p):
    return random.random() < p

def draw(G, node_color="lightblue", edge_color = "black"):
    pos=nx.spring_layout(G)
    plt.figure(figsize=(20,20))
    nx.draw(G, pos, with_labels=True, node_color=node_color, edge_color=edge_color, node_size=500)
    plt.show()

In [5]:
# choose the node to which attach the new edge
def choose_attachment(G, event="node_event"):

    # calculate the probability of a node to be chosen (deg(v)^gamma)/(sum_v(deg(v)^gamma))
    prob = {}
    for n in G.nodes():
        deg_n = pow(len(G.edges(n)), GAMMA)
        
        tot_deg = sum([ pow(len(G.edges(i)), GAMMA) for i in G.nodes()])
        if(n!=N or event=="edge_event"):
            prob[n] = deg_n/tot_deg
                
   #print(prob)
    # extract a node 
    done = False
    while(not done):
        for n in G.nodes():
            if(n!=N or event=="edge_event"):
                if(choose(prob[n])):
                    return n

In [6]:
# networks generation
frames = []
for rep in range(NUM_NETWORKS):

    N = 0 # next node label

    Graphs = []  # contains the evolution of the original graph by event
    G = nx.Graph()

    # create the startin graph
    G.add_node(N)
    Graphs.append(G.copy())
    N += 1

    G.add_node(N)
    G.add_edge(N-1, N)
    Graphs.append(G.copy())
    N += 1

    for e in range(NUM_EVENTS-2):
        
        # Node event
        if(choose(P) or isComplete(G)):
            # node_event
            G.add_node(N)
            
            # chose the node to which attach the new edge
            n1 = choose_attachment(G)
            G.add_edge(N, n1)
            
            N += 1
            
        # Edge event
        else:
            n1, n2 = 0, 0
            # choose the nodes to which attach the new edge
            while(n1==n2 or (n1,n2) in G.edges()):
                n1 = choose_attachment(G, "edge_event")
                n2 = choose_attachment(G, "edge_event")            
                
            G.add_edge(n1, n2)
            
            
        Graphs.append(G.copy())
        
        sys.stdout.write('\r Rep: '+str(rep)+"/"+str(NUM_NETWORKS)+", events: "+str(e+2)+"/"+str(NUM_EVENTS))
            
    # draw(G)

    df = pd.DataFrame()
    event = 0
    for curr_G in Graphs:

        col = [0 for i in range(len(G.nodes()))]
        for node, deg in curr_G.degree():
            col[node] = deg
        
        df[event] = col
        event += 1
        
    frames.append(df)
    
df = pd.concat(frames)

 Rep: 99/100, events: 499/500

In [7]:
# set starting event for each row 
start_e = []
for index, row in df.iterrows():
    row = np.array(row)
    e = 0
    while row[e]==0:
        e+=1 
    
    start_e.append(e)

df["start_event"] = start_e
df

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,start_event
0,0,1,1,1,1,1,1,1,1,1,...,17,17,17,17,17,17,17,17,17,1
1,0,1,2,3,3,4,4,5,6,6,...,103,103,103,103,103,103,103,103,103,1
2,0,0,1,1,1,1,1,1,1,1,...,5,5,5,5,5,5,5,5,5,2
3,0,0,0,1,2,2,3,3,3,3,...,40,40,40,40,40,40,40,40,40,3
4,0,0,0,0,1,1,1,1,1,1,...,11,11,11,11,11,11,11,11,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,495
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,496
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,497
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,498


In [8]:
df.to_csv("Barabasi-Albert.csv", index=0)
#df.to_csv("Chung-Li_p05_b1.csv", index=0)
# df.to_csv("Chung-Li_p05_b05.csv", index=0)